In [ ]:
# Python 3

import numpy as np
import os

In [ ]:
folder_name = "10by10grid_car_truck"
num_row = 10
num_col = 10
O_prob = 0.4
D_prob = 0.4
cap_car = 2000
cap_truck = 1200
length = 0.5
rhoj_car = 200
rhoj_truck = 120
num_lane = 2
spd_lb = 15
spd_ub = 55

# 1 truck = how many cars in node model
conv_factor = 2.0

assign_horizon = 10
demand_car = 50
demand_truck = demand_car//20

np.random.seed(0)

In [ ]:
link_type = 'CTM'
node_type = 'FWJ'
connector_type = 'PQ'
O_type = 'DMOND'
D_type = 'DMDND'

In [ ]:
file_attr = "MNM_input_"
node_file = file_attr + "node"
link_file = file_attr + "link"
snap_file = "Snap_graph"
od_file = file_attr + "od"
demand_file = file_attr + 'demand'

In [ ]:
def add_node_str(node_num, type_str):
    return ' '.join([str(e) for e in [node_num, type_str, conv_factor]]) + '\n'

In [ ]:
def add_snap_str(link_num, from_node, to_node):
    return ' '.join([str(e) for e in [link_num, from_node, to_node]]) + '\n'

In [ ]:
def add_link_str(link_num, link_type):
    # 1 PQ 1 99999 99999 99999 1 99999 99999 99999 2
    # 2 CTM 0.55 35 2200 200 2 25 1200 120 2
    if link_type != 'PQ':
        spd_car = np.round(np.random.random_sample() * (spd_ub - spd_lb)  + np.float(spd_lb))
        spd_truck = np.round(spd_car * 0.8)
        return ' '.join([str(e) for e in [link_num, link_type, length, spd_car, cap_car, rhoj_car, 
                                          num_lane, spd_truck, cap_truck, rhoj_truck, conv_factor]]) + '\n'
    else:
        return str(link_num) + ' PQ 1 99999 99999 99999 1 99999 99999 99999 2\n'

In [ ]:
def add_OD(od_id, node_id):
    return str(od_id) + ' ' + str(node_id) + '\n'

In [ ]:
link_str_list = []
node_str_list = []
snap_str_list = []
O_dict = dict()
D_dict = dict()
O_str_list = []
D_str_list = []

node_counter = 1
link_counter = 1
O_counter = 1
D_counter = 1

for i in range(num_row):
    for j in range(num_col):
        node_str_list.append(add_node_str(node_counter, node_type))
        if (j != num_col - 1):
            node1 = num_col * i + j + 1
            node2 = node1 + 1
            link_str_list.append(add_link_str(link_counter, link_type))
            snap_str_list.append(add_snap_str(link_counter, node1, node2))
            link_counter += 1
            link_str_list.append(add_link_str(link_counter, link_type))
            snap_str_list.append(add_snap_str(link_counter, node2, node1))   
            link_counter += 1
        if(i != num_row - 1):
            node1 = num_col * i + j + 1
            node2 = node1 + num_col
            link_str_list.append(add_link_str(link_counter, link_type))
            snap_str_list.append(add_snap_str(link_counter, node1, node2))
            link_counter += 1
            link_str_list.append(add_link_str(link_counter, link_type))
            snap_str_list.append(add_snap_str(link_counter, node2, node1)) 
            link_counter += 1
        node_counter += 1
            
for i in range(num_row):
    for j in range(num_col):
        node1 = num_col * i + j + 1
        if (np.random.random_sample() < O_prob or (i == 0 and j ==0)):
            node_str_list.append(add_node_str(node_counter, O_type))
            O_dict[O_counter] = node1
            O_str_list.append(add_OD(O_counter, node_counter))
            link_str_list.append(add_link_str(link_counter, connector_type))
            snap_str_list.append(add_snap_str(link_counter, node_counter, node1)) 
            link_counter += 1
            node_counter += 1
            O_counter += 1
        if (np.random.random_sample() < D_prob or (i == num_row -1 and j == num_col - 1)):
            node_str_list.append(add_node_str(node_counter, D_type))
            D_dict[D_counter] = node1
            D_str_list.append(add_OD(D_counter, node_counter))
            link_str_list.append(add_link_str(link_counter, connector_type))
            snap_str_list.append(add_snap_str(link_counter, node1, node_counter)) 
            link_counter += 1
            node_counter += 1
            D_counter += 1


In [ ]:
def add_demand_str(O, D):
    l = [str(O), str(D)]
    d_car = [str(demand_car) for i in range(assign_horizon)]
    d_truck = [str(demand_truck) for i in range(assign_horizon)]
    l = l + d_car + d_truck
    return ' '.join(l) + '\n'

In [ ]:
# Origin_ID Destination_ID <demand by interval>
demand_str_list = []

for O in O_dict:
    for D in D_dict:
        if O_dict[O] != D_dict[D]:
            demand_str_list.append(add_demand_str(O, D))

In [ ]:
if not os.path.isdir(folder_name):
    os.makedirs(folder_name)

f = open(os.path.join(folder_name, link_file), 'w')
f.write("#ID Type LEN(mile) FFS_car(mile/h) Cap_car(v/hour) RHOJ_car(v/miles) \
        Lane FFS_truck(mile/h) Cap_truck(v/hour) RHOJ_truck(v/miles) Convert_factor(1)\n")
for line in link_str_list:
    f.write(line)
f.close()

f = open(os.path.join(folder_name, node_file), 'w')
f.write('#ID Type Convert_factor(only for Inout node)\n')
for line in node_str_list:
    f.write(line)
f.close()

f = open(os.path.join(folder_name, snap_file), 'w')
f.write('# EdgeID FromNodeId ToNodeId\n')
for line in snap_str_list:
    f.write(line)
f.close()

f = open(os.path.join(folder_name, od_file), 'w')
f.write('#Origin_ID <-> node_ID\n')
for line in O_str_list:
    f.write(line)
f.write('#Dest_ID <-> node_ID\n')
for line in D_str_list:
    f.write(line)
f.close()

f = open(os.path.join(folder_name, demand_file), 'w')
f.write('#Origin_ID Destination_ID <car demand by interval> <truck demand by interval>\n')
for line in demand_str_list:
    f.write(line)
f.close()

In [ ]:
print('maximum_interval:', assign_horizon)
print('num_of_link:', len(link_str_list))
print('num_of_node:', len(node_str_list))
print('num_of_O:', len(O_dict))
print('num_of_D:', len(D_dict))
print('num_of_od:', len(demand_str_list))